<div style="text-align: right"><b> Jessenia Piza Londoño & Paula Lorena López Romero. </b></div>

# Proyecto 4: Bot Hablador

En este proyecto trabajaremos el Automatic speech recognition (ASR), donde haremos reconocimiento automático de voz. El objetivo principal es crear un bot hablador, donde a partir de la utilización de transformers, podemos grabar un audio y a partir de este, generar una respuesta del bot. Se utilizarán diferentes modelos de transformers de HuggingFace, con el fin de hacer el reconocimiento de voz y el diálogo con el bot de mejor manera. A lo largo del notebook veremos las limitaciones que tenemos y los resultados obtenidos.

Primero realizaremos el procedimiento paso a paso y posteriormente la interfaz gráfica de la aplicación funcional.

Importamos las librerías a utilizar.

In [1]:
import torch
import gradio as gr
import IPython.display as ipd
import sounddevice as sd
import soundfile as sf
import librosa
from huggingsound import SpeechRecognitionModel
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, AutoModelForCausalLM

Realizamos el pipeline donde grabamos el audio que utilizaremos y podemos visualizar y escuchar lo que acabamos de grabar.

In [2]:
samplerate = 48000  
duration = 5 # seconds
filename = 'grabacion.wav'
print("grabando...")
mydata = sd.rec(int(samplerate * duration), samplerate=samplerate,
    channels=1, blocking=True)
print("grabación terminada")
sd.wait()
sf.write(filename, mydata, samplerate)
#reading the voice commands
samples, sample_rate = librosa.load(filename , sr = 48000)
samples = librosa.resample(samples, sample_rate, 16000)
ipd.Audio(samples,rate=16000)

grabando...
grabación terminada


## Parte 1: De voz a texto

Primero, realizaremos `wav2vec` para obtener el audio en forma de vector y posteriormente pasarlo a texto. Para esto, tomamos el modelo `jonatasgrosman/wav2vec2-large-xlsr-53-spanish` de HuggingFace. Notamos que podemos utilizar este modelo a partir de `huggingsound` con `SpeechRecognitionModel` y lo cargamos.

In [3]:
model_audio = SpeechRecognitionModel("jonatasgrosman/wav2vec2-large-xlsr-53-spanish")

05/24/2022 00:52:51 - INFO - huggingsound.speech_recognition.model - Loading model...


Tomamos el path del audio anteriormente grabado y utilizamos el modelo anterior para transcribir el audio.

In [4]:
audio_paths = ["grabacion.wav"]
transcriptions = model_audio.transcribe(audio_paths)

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.24s/it]


Obtenemos el resultado y notamos que ni las tildes ni los signos de puntuación los coloca de manera correcta.

In [5]:
transcription = transcriptions[0]['transcription']
transcription

'como te llamas'

## Parte 2: Traducción al inglés

Ahora, con la transcripción previamente observada, realizamos la traducción de esta al inglés. En este caso, utilizamos un transformer de `HuggingFace` llamado `Helsinki-NLP/opus-mt-es-en`, el cual traduce únicamente del español al inglés, creamos el tokenizador y el modelo.

In [6]:
tokenizer_trans = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-es-en")
model_trans = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-es-en")

Generamos los tokens a partir del texto transcrito y decodificamos la transcripción creada por el modelo con los tokens, de esta manera, obtenemos el texto traducido. Notamos que, los signos de puntuación esta vez los coloca de manera adecuada al contrario que en el español.

In [7]:
tokens_t = tokenizer_trans(transcription, return_tensors="pt", padding=True)
translated = model_trans.generate(**tokens_t)
text_translated = [tokenizer_trans.decode(t, skip_special_tokens=True) for t in translated][0]
text_translated

"What's your name?"

## Parte 3: Bot conversacional

Ahora bien, con respecto a la pregunta o afirmación dada por el usuario, vamos a generar una respuesta a partir de un bot. En este caso utilizaremos `microsoft/DialoGPT-medium` para crear el diálogo entre el usuario y el bot. En este caso, el modelo funciona de mejor manera con un texto en inglés, por ende, se utilizará el texto traducido anteriormente. Primero, creamos el tokenizador y el modelo.

In [8]:
tokenizer_bot = AutoTokenizer.from_pretrained("microsoft/DialoGPT-medium")
model_bot = AutoModelForCausalLM.from_pretrained("microsoft/DialoGPT-medium")

Con el texto traducido y el modelo anterior generamos el vector con la respuestas, posteriormente utilizamos el decoder para traducir este vector a texto y visualizamos el correcto funcionamiento del bot.

In [9]:
bot_input_ids = tokenizer_bot.encode(text_translated + tokenizer_bot.eos_token, return_tensors='pt')
answer = model_bot.generate(bot_input_ids, max_length=1000, pad_token_id=tokenizer_bot.eos_token_id)
answer_decode = tokenizer_bot.decode(answer[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)
answer_decode

"I'm not sure, but I think it's a reference to the movie The Big Lebowski."

## Parte 4: Traducción del inglés al español

Por último, como el audio que se ingresa es en español, la respuesta debe ser acorde a este. Por ende, utilizamos un transformer que toma el texto en inglés y lo traduce al español, en este caso tomamos `Helsinki-NLP/opus-mt-en-es` de `HuggingFace`. Nuevamente, creamos el tokenizador y el modelo.

In [10]:
tokenizer_tr = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-es")
model_tr = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-en-es")

Generamos la respuesta anteriormente visualizada pero traducida. Así, terminaríamos el procedimiento del bot conversacional.

In [11]:
answer_translated = model_tr.generate(**tokenizer_tr(answer_decode, return_tensors="pt", padding=True))
answer_translated = [tokenizer_tr.decode(t, skip_special_tokens=True) for t in answer_translated][0]
answer_translated

'No estoy seguro, pero creo que es una referencia a la película El Gran Lebowski.'

## Parte 5: Interfaz Gráfica

Ahora, para una mejor experencia con el usuario, realizaremos una interfaz gráfica con Gradio, en esta grabamos el audio, visualizamos su transcripción y recibimos la respuesta del bot de manera escrita (esto debido a que no se encontraron transformers buenos en HuggingFace que pudieran simular la voz del bot en español). Además, creamos una variable global que mantiene el historial de la conversación, para que, mientras se esté en la interfaz, se tenga sólo una conversación (para iniciar una nueva conversación, vuelva a iniciar el historial y la interfaz de gradio).

In [12]:
chat_history_ids = torch.tensor([])

Creamos la función con todo el procedimiento previamente específicado. Con esto, recolectamos el audio con gradio, transcribimos, traducimos, obtenemos la respuesta del bot y volvemos a traducir esta.

In [14]:
def chat_bot(audio):
    global chat_history_ids
    audio_paths = [audio]
    transcriptions = model_audio.transcribe(audio_paths)
    transcription = transcriptions[0]['transcription']
    tokens_t = tokenizer_trans(transcription, return_tensors="pt", padding=True)
    translated = model_trans.generate(**tokens_t)
    text_translated = [tokenizer_trans.decode(t, skip_special_tokens=True) for t in translated][0]
    
    new_user_input_ids = tokenizer_bot.encode(text_translated + tokenizer_bot.eos_token, return_tensors='pt')
    bot_input_ids = torch.cat([chat_history_ids, new_user_input_ids], dim=-1) if chat_history_ids.shape[0] > 0 else new_user_input_ids
    chat_history_ids = model_bot.generate(bot_input_ids, max_length=1000, pad_token_id=tokenizer_bot.eos_token_id)
    
    answer_decode = tokenizer_bot.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)
    answer_translated = model_tr.generate(**tokenizer_tr(answer_decode, return_tensors="pt", padding=True))
    answer_translated = [tokenizer_tr.decode(t, skip_special_tokens=True) for t in answer_translated][0]
    return transcription, answer_translated

Observemos la interfaz.

In [16]:
gr.Interface(
    fn=chat_bot, 
    inputs=gr.inputs.Audio(source="microphone", type="filepath"), 
    outputs=[gr.outputs.Textbox(label = 'Transcription'),
             gr.outputs.Textbox(label = 'Bot Answer')],
    title = 'Bot Hablador').launch()

c:\users\jesse\appdata\local\programs\python\python39\lib\site-packages\gradio\deprecation.py:40: UserWarning: `optional` parameter is deprecated, and it has no effect
  warnings.warn(value)
c:\users\jesse\appdata\local\programs\python\python39\lib\site-packages\gradio\deprecation.py:40: UserWarning: The 'type' parameter has been deprecated. Use the Number component instead.
  warnings.warn(value)


Running on local URL:  http://127.0.0.1:7861/

To create a public link, set `share=True` in `launch()`.


(<gradio.routes.App at 0x204112be4c0>, 'http://127.0.0.1:7861/', None)

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.44s/it]


# Conclusiones y observaciones

- Al realizar procedimientos y procesamiento a partir del reconocimiento de voz es muy importante tener buena vocalización e intentar que todo se escuche perfectamente. Esto debido a que, si se recibe algo que no se quería decir, es muy probable que el resultado no sea el esperado.


- Los transformers nos permiten trabajar con la traducción de textos, el reconocimiento de voz, entre otras cosas. Sin embargo, en los modelos que trabajan en español, es complicado que el audio escuchado lo tome de manera correcta, esto debido a que no reconoce bien cuándo se acentúan o no ciertas palabras. Lo mismo sucede con la puntuación del texto, así que, cuando no toma algunos signos de puntuación en el audio, la respuesta que da el bot es incoherente con lo que el usuario quiso expresar.


- Los transformers que trabajan en inglés manejan de manera adecuada entre sí, sin embargo, notamos durante el desarrollo del proyecto que el transformer que creaba la respuesta del bot muchas veces se quedaba en una oración ya dicha. Esto puede ser debido a que, cuando se toma el historial de la conversación, toma desde el inicio de esta lo que hace que el bot se confunda sobre el tema que se está hablando. Llegamos a esta conclusión porque cuando se tiene una conversación entre dos personas, muchas veces el contexto puede variar y lo del principio ya no importaría. Sin embargo, sería bueno analizar en qué casos sería correcto descartar algunas oraciones en el historial y en qué otros no.


- Por último, para mejorar en este proyecto tendríamos dos cosas principales: la puntuación y gramática correctas a la hora de pasar de audio a texto y el correcto manejo del historial en la conversación, además, en la interfaz gráfica poder colocar un botón donde se reinicie este.

<div style="text-align: right"><b> Jessenia Piza Londoño & Paula Lorena López Romero. </b></div>